In [ ]:
!pip install nudged

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import glob
import json
import psutil
import random
import os
import time
import sys
import math
import scipy.interpolate
import scipy.sparse
from tqdm import tqdm
from contextlib import contextmanager
import matplotlib.pylab as plt
import shapely
import matplotlib
import nudged
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import pickle
from shapely.geometry import shape, GeometryCollection, Polygon, MultiPolygon
from shapely.affinity import affine_transform
from PIL import Image, ImageOps
from skimage.morphology import convex_hull_image
from shapely.geometry import Point
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.spatial.distance import cdist
from pathlib import Path

In [ ]:
#Provided GitHub functions
!cp -r ../input/indoor-location-competition-20-git/* ./
import io_f
import visualize_f 
import compute_f
import main  

# FUNCTIONS

* ### Additional

In [ ]:
floors_key_assign = {'1F':0, '2F':1, '3F':2, '4F':3, '5F':4, '6F':5, '7F':6, '8F':7, '9F':8, 
          'B':-1, 'B1':-1,'B2':-2, 'B3':-3, 'BF':-1, 'BM':-1, 
          'F1':0, 'F2':1, 'F3':2, 'F4':3, 'F5':4, 'F6':5,'F7':6, 'F8':7, 'F9':8, 'F10':9,
          'G':0, 'LG1':-1, 'LG2':-2, 
          "L1":0,"L2":1,"L3":2,"L4":3,"L5":4,"L6":5,"L7":6,"L8":7,"L9":8,"L10":9,'L11':10, 
          'LM':0, 'M':0, 'P1':-1, 'P2':-2,
          "地下一层":-1,"一层":0,"二层":1,"三层":2,"四层":3}

def get_site_floor_from_id(site, floor_id):
    site_p = "../input/indoor-location-navigation/train/"+site
    floor_p = "/*"
    site_floor_names = []
    floors_files = sorted(glob.glob(site_p + floor_p))
    for f, floor in enumerate(floors_files):
        floor_name = floor.split('/')[-1]
        site_floor_names.append(floor_name)
    for key_f in floors_key_assign:
        if key_f in site_floor_names:
            if floors_key_assign[key_f] == floor_id:
                return key_f
    return floor_id

def highlight_dif(x):
    r = 'red'
    g = 'green'

    m1 = x['f'] != x['pred_f']
    m2 = x['f'] == x['pred_f']
    
    m3 = x['x'] != x['pred_x']
    m4 = x['x'] == x['pred_x']
    
    m5 = x['y'] != x['pred_y']
    m6 = x['y'] == x['pred_y']

    df1 = pd.DataFrame('color: ', index=x.index, columns=x.columns)
    #rewrite values by boolean masks
    df1['pred_f'] = np.where(m1, 'color: {}'.format(r), df1['pred_f'])
    df1['pred_f'] = np.where(m2, 'color: {}'.format(g), df1['pred_f'])
    
    df1['pred_x'] = np.where(m3, 'color: {}'.format(r), df1['pred_x'])
    df1['pred_x'] = np.where(m4, 'color: {}'.format(g), df1['pred_x'])
    
    df1['pred_y'] = np.where(m5, 'color: {}'.format(r), df1['pred_y'])
    df1['pred_y'] = np.where(m6, 'color: {}'.format(g), df1['pred_y'])
    return df1

def highlight_eval(x):
    r = 'red'
    g = 'green'
    y = 'yellow'

    m1 = x['floor_knn_acc'] > x['floor_fix_acc']
    m2 = x['floor_knn_acc'] == x['floor_fix_acc']
    m3 = x['floor_knn_acc'] < x['floor_fix_acc']
    
    m4 = x['wp_knn_mpe'] < x['wp_fix_mpe']
    m5 = x['wp_knn_mpe'] == x['wp_fix_mpe']
    m6 = x['wp_knn_mpe'] > x['wp_fix_mpe']
    
    m7 = x['wp_knn_acc'] > x['wp_fix_acc']
    m8 = x['wp_knn_acc'] == x['wp_fix_acc']
    m9 = x['wp_knn_acc'] < x['wp_fix_acc']
    
    m10 = x['wp_knn_rmse'] < x['wp_fix_rmse']
    m11 = x['wp_knn_rmse'] == x['wp_fix_rmse']
    m12 = x['wp_knn_rmse'] > x['wp_fix_rmse']
    
    
    df1 = pd.DataFrame('color: ', index=x.index, columns=x.columns)
    #rewrite values by boolean masks
    df1['floor_fix_acc'] = np.where(m1, 'color: {}'.format(r), df1['floor_fix_acc'])
    df1['floor_fix_acc'] = np.where(m2, 'color: {}'.format(y), df1['floor_fix_acc'])
    df1['floor_fix_acc'] = np.where(m3, 'color: {}'.format(g), df1['floor_fix_acc'])
    
    df1['wp_fix_mpe'] = np.where(m4, 'color: {}'.format(r), df1['wp_fix_mpe'])
    df1['wp_fix_mpe'] = np.where(m5, 'color: {}'.format(y), df1['wp_fix_mpe'])
    df1['wp_fix_mpe'] = np.where(m6, 'color: {}'.format(g), df1['wp_fix_mpe'])
    
    df1['wp_fix_acc'] = np.where(m7, 'color: {}'.format(r), df1['wp_fix_acc'])
    df1['wp_fix_acc'] = np.where(m8, 'color: {}'.format(y), df1['wp_fix_acc'])
    df1['wp_fix_acc'] = np.where(m9, 'color: {}'.format(g), df1['wp_fix_acc'])
    
    df1['wp_fix_rmse'] = np.where(m10, 'color: {}'.format(r), df1['wp_fix_rmse'])
    df1['wp_fix_rmse'] = np.where(m11, 'color: {}'.format(y), df1['wp_fix_rmse'])
    df1['wp_fix_rmse'] = np.where(m12, 'color: {}'.format(g), df1['wp_fix_rmse'])
    return df1

* ### Plot on Map

In [ ]:
def plot_floor(
    site,
    floorNo,
    paths_wp_df=None,
    grid_wp=None,
    grid_gen=None,
    display_corridors=True,
    display_map=True,
    w_size=8,
    w_color='grey',
    g_size=4,
    g_color='grey',
    
    base="../input/indoor-location-navigation",
    base_pol = "../input/indoor-location-navigation-scaled-geojson/scaled_geojson",
):
    map_floor = floorNo
    # Prepare width_meter & height_meter (taken from the .json file)
    floor_plan_filename = f"{base}/metadata/{site}/{map_floor}/floor_image.png"
    json_plan_filename = f"{base}/metadata/{site}/{map_floor}/floor_info.json"
    with open(json_plan_filename) as json_file:
        json_data = json.load(json_file)
    width_meter = json_data["map_info"]["width"]
    height_meter = json_data["map_info"]["height"]
    floor_img = f"{base}/metadata/{site}/{map_floor}/floor_image.png"
    
    file = f"{base_pol}/{site}/{map_floor}/shapely_geometry.pkl"
    with open(file, 'rb') as f:
        geometry = pickle.load(f)
    
    m_opacity=0.8
    mode='lines + markers'
    
    #Display
    fig = go.Figure()
    
    if paths_wp_df is not None:
        for path, path_data in paths_wp_df.groupby("path"):
            fig.add_trace(
                go.Scattergl(
                    x=path_data["x"],
                    y=path_data["y"],
                    mode=mode,
                    marker_size=10,
                    name=path,
                    visible='legendonly'
                ))

    if grid_wp is not None:
        fig.add_trace(
            go.Scattergl(
                name="Waypoints Grid",
                x=grid_wp[:, 0],
                y=grid_wp[:, 1],
                mode='markers',
                opacity=m_opacity,
                marker_symbol=3,
                marker_color=w_color,
                marker_size=w_size,
                visible='legendonly'

            ))
    if grid_gen is not None:
        fig.add_trace(
            go.Scattergl(
                name="Generated Grid",
                x=grid_gen[:, 0],
                y=grid_gen[:, 1],
                mode='markers',
                opacity=m_opacity,
                marker_symbol=3,
                marker_color=g_color,
                marker_size=g_size,
                visible='legendonly'
            ))
    
    if display_map:
        floor_plan = Image.open(floor_img)
        fig.update_layout(images=[
            go.layout.Image(
                source=floor_plan,
                xref="x",
                yref="y",
                x=0,
                y=height_meter,
                sizex=width_meter,
                sizey=height_meter,
                sizing="contain",
                opacity=0.5,
                layer="below",
            )
        ])
    
    if display_corridors:
        for coord in extract_coords_from_polygon(geometry):
            x, y = coord
            fig.add_trace(
                go.Scattergl(
                    x=x,
                    y=y,
                    opacity=0.6,
                    text=None,
                    marker_color='red',
                    hoverinfo='skip',
                    showlegend=False
                ))
    
    # configure
    fig.update_xaxes(autorange=False, range=[0, width_meter])
    fig.update_yaxes(autorange=False, range=[0, height_meter], scaleanchor="x", scaleratio=1)
    fig.update_layout(
        title=go.layout.Title(
            text="Site: %s Floor: %s" % (site, map_floor),
            xref="paper",
            x=0,
        ),
        autosize=True,
        width=800,
        height=800 * height_meter / width_meter,
        template="plotly_white",
    )

    return fig

* ### Generate Grid from Map Geometry

In [ ]:
def generate_grid_from_map(stite_check, floor_check, step = 2):
    base_pol = "../input/indoor-location-navigation-scaled-geojson/scaled_geojson"
    file = f"{base_pol}/{stite_check}/{floor_check}/shapely_geometry.pkl"
    with open(file, 'rb') as f:
        corridor = pickle.load(f)
    
    x_min, y_min, x_max, y_max = get_bounding_box(corridor)

    x_range = range(math.ceil(x_min), math.floor(x_max), step)
    y_range = range(math.ceil(y_min), math.floor(y_max), step)

    valid_coords = []

    for x in x_range:
        for y in y_range:
            if Point(x, y).within(corridor):
                valid_coords.append([x, y])

    return np.array(valid_coords)

def get_bounding_box(shapes):
    """
    To extract bounding box from Polygon
    """
    x_min = 10000
    y_min = 10000
    x_max = 0
    y_max = 0
    
    if type(shapes) == Polygon:
            shapes = [shapes]
    for shape in shapes:
        x, y = shape.exterior.xy
        x_min = min(min(x), x_min)
        y_min = min(min(y), y_min)
        x_max = max(max(x), x_max)
        y_max = max(max(y), y_max)
    return x_min, y_min, x_max, y_max

def extract_coords_from_polygon(polygon):
    coords = []
    if type(polygon) == MultiPolygon:
        polygons = polygon.geoms
    else:
        polygons = [polygon]

    for polygon in polygons:
        x, y = polygon.exterior.xy
        coords.append((np.array(x), np.array(y)))
        for interior in polygon.interiors:
            x, y = interior.xy
            coords.append((np.array(x), np.array(y)))

    return coords

* ### Generate Grid from WP Paths

In [ ]:
def generate_grid_from_wp(site:str=None,floor:str=None, start:int=None, stop:int=None):
    site_p = "../input/indoor-location-navigation/train/*"
    floor_p = "/*"
    path_p = "/*"
        
    if (site is not None) & (floor is None):
        site_p = "../input/indoor-location-navigation/train/" + site

    if (site is not None) & (floor is not None):
        site_p = "../input/indoor-location-navigation/train/" + site
        floor_p = "/" + floor
    
    #Go through sites
    sites = sorted(glob.glob(site_p))
    
    if (start is not None) & (stop is not None):
        sites = sites[start:stop]
    
    #Create DF of all waypoints in whole site
    main_wp_df = pd.DataFrame(columns = ['site', 'floor', 'x', 'y'])
    for s, site in enumerate(sites):
        #print site info
        site_id = site.split('/')[-1]

        #Create DF of all waypoints in whole site
        site_wp_df = pd.DataFrame(columns = ['site', 'floor', 'x', 'y'])
        total_path = 0

        #Go through floors
        floors = sorted(glob.glob(site + floor_p))  
        for f, floor in enumerate(floors):
            #print floor info
            floor_id = floor.split('/')[-1]

            #Create DF of all waypoints on each floor
            wp_floor_df = pd.DataFrame(columns = ['site', 'floor', 'x', 'y'])

            #Go through paths
            paths = sorted(glob.glob(floor + path_p))   
            for p, path in enumerate(paths):                
                total_path += len(paths)

                #print path info
                path_id = path.split('/')[-1].split('.')[0]


                #Exstract All Paths Waypoints
                path_wp = io_f.read_data_file(path).waypoint           
                for wp in path_wp:
                    new_row = {'site':site_id, 'floor':floor_id, 'x':wp[1], 'y':wp[2]}
                    wp_floor_df = wp_floor_df.append(new_row, ignore_index=True)

                #Drop duplicate waypoints
                wp_floor_df = wp_floor_df.drop_duplicates(subset=['x', 'y'], keep='last')

            #load to site wp df
            site_wp_df = site_wp_df.append(wp_floor_df, ignore_index=True)

        main_wp_df = main_wp_df.append(site_wp_df, ignore_index=True)

    site_floor_list = main_wp_df.groupby(['site', 'floor'],as_index=False).agg({'x': 'count'})    
    if (site is not None) | (floor is not None):
        return main_wp_df, site_floor_list
    else:
        return main_wp_df

* ### Get All Path from Floor

In [ ]:
def get_floor_paths(stite_check,floor_check):
    base = '../input/indoor-location-navigation/train'
    floor_paths_files = sorted(glob.glob("{}/{}/{}/*".format(base,stite_check,floor_check)))
    floor_paths_df = pd.DataFrame(columns = ['site', 'floor', 'path', 'timestamp', 'x', 'y'])
    for floor_paths_file in floor_paths_files:
        path_id = floor_paths_file.split('/')[-1].split('.')[0]
        #Exstract All Paths Waypoints
        path_wp = io_f.read_data_file(floor_paths_file).waypoint
        path_wp_df = pd.DataFrame(columns = ['site', 'floor', 'path', 'timestamp', 'x', 'y'])
        for wp in path_wp:
                new_row = {'site':stite_check, 'floor':floor_check, 'path':path_id, 'timestamp':wp[0], 'x':wp[1], 'y':wp[2]}
                path_wp_df = path_wp_df.append(new_row, ignore_index=True)

        floor_paths_df = floor_paths_df.append(path_wp_df, ignore_index=True)
    return (floor_paths_df)

* ### Cost Minimisation

In [ ]:
def compute_rel_positions(acce_datas, ahrs_datas, posi_datas):
    step_timestamps, step_indexs, step_acce_max_mins = compute_f.compute_steps(acce_datas)
    headings = compute_f.compute_headings(ahrs_datas)
    stride_lengths = compute_f.compute_stride_length(step_acce_max_mins)
    step_headings = compute_f.compute_step_heading(step_timestamps, headings)
    rel_positions = compute_f.compute_rel_positions(stride_lengths, step_headings)
    return rel_positions

def correct_path(site, path_df, alpha_mod=2, beta_mod=2):
    site_p = "../input/indoor-location-navigation/train/"
    path = path_df['path'].values[0]
    floor = path_df['floor'].values[0]
    T_ref  = path_df['timestamp'].values
    xy_hat = path_df[['x', 'y']].values
    
    site_floor = get_site_floor_from_id(site,floor)
    
    example = io_f.read_data_file(f'{site_p}/{site}/{site_floor}/{path}.txt')
    rel_positions = compute_rel_positions(example.acce, example.ahrs, path_df[['timestamp','x','y']].to_numpy())
    '''   
    posi_datas = pd.DataFrame(columns=['timestamp','x', 'y'])
    posi_datas['timestamp'] = step_positions[:, 0]
    posi_datas['x'] = step_positions[:, 1]
    posi_datas['y'] = step_positions[:, 2]
    posi_datas['floor'] = floor
    posi_datas['path'] = path'''
    
    if T_ref[-1] > rel_positions[-1, 0]:
        rel_positions = [np.array([[0, 0, 0]]), rel_positions, np.array([[T_ref[-1], 0, 0]])]
    else:
        rel_positions = [np.array([[0, 0, 0]]), rel_positions]
    rel_positions = np.concatenate(rel_positions)
    
    T_rel = rel_positions[:, 0]
    delta_xy_hat = np.diff(scipy.interpolate.interp1d(T_rel, np.cumsum(rel_positions[:, 1:3], axis=0), axis=0)(T_ref), axis=0)

    N = xy_hat.shape[0]
    delta_t = np.diff(T_ref)
    alpha = (8.1)**(-alpha_mod) * np.ones(N)
    beta  = (0.3 + 0.3 * 1e-3 * delta_t)**(-beta_mod)
    A = scipy.sparse.spdiags(alpha, [0], N, N)
    B = scipy.sparse.spdiags( beta, [0], N-1, N-1)
    D = scipy.sparse.spdiags(np.stack([-np.ones(N), np.ones(N)]), [0, 1], N-1, N)

    Q = A + (D.T @ B @ D)
    c = (A @ xy_hat) + (D.T @ (B @ delta_xy_hat))
    xy_star = scipy.sparse.linalg.spsolve(Q, c)
    
    path_df['x'] = xy_star[:, 0]
    path_df['y'] = xy_star[:, 1]
    
    return path_df

* ### Grid Generation

In [ ]:
def generate_grid_for_floor(stite_check, floor_check, step, all_paths=False):
    #collect all waypoints on floor
    if all_paths:
        floor_paths_df = get_floor_paths(stite_check,floor_check)
    #grid from waypoints
    floor_wp_grid_df, site_floor_list = generate_grid_from_wp(stite_check, floor_check)
    floor_wp_grid = floor_wp_grid_df[['x','y']].to_numpy() #for ploting
    #generate grid based on map
    floor_map_grid = generate_grid_from_map(stite_check, floor_check, step)
    #snap to grid
    #fixed_floor_paths_df = snap_to_grid(floor_paths_df, floor_map_grid, 2)
    if all_paths:
        return floor_paths_df, floor_wp_grid, floor_map_grid
    else:
        return floor_wp_grid, floor_map_grid

# POST-PROCESSING

### Snap to Grid

In [ ]:
def snap_to_grid(wp, grid, threshold):
    wp = find_closest_grid(wp, grid)
    wp['dist'] = np.sqrt((wp.x-wp.x_)**2 + (wp.y-wp.y_)**2)
    
    wp = snap_points_to_grid(wp, threshold=threshold)
    wp = wp[['site','path', 'floor', 'timestamp','_x_','_y_']].rename(columns={'_x_':'x', '_y_':'y'})
    return wp

def add_xy(df):
    df['xy'] = [(x, y) for x,y in zip(df['x'], df['y'])]
    return df
def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]
def find_closest_grid(wp, grid):
    grid_df = pd.DataFrame(grid, columns = ['x','y'])
    wp = add_xy(wp)
    grid_df = add_xy(grid_df)
    ds = []
    for (site, myfloor), d in wp.groupby(['site','floor']):
        true_floor_locs = grid_df.reset_index(drop=True)
        if len(true_floor_locs) == 0:
            print(f'Skipping {site} {myfloor}')
            continue
        d['matched_point'] = [closest_point(x, list(true_floor_locs['xy'])) for x in d['xy']]
        d['x_'] = d['matched_point'].apply(lambda x: x[0])
        d['y_'] = d['matched_point'].apply(lambda x: x[1])
        ds.append(d)

    wp = pd.concat(ds)
    
    return(wp)
def snap_points_to_grid(sub, threshold):
    """
    Snap to grid if within a threshold.
    
    x, y are the predicted points.
    x_, y_ are the closest grid points.
    _x_, _y_ are the new predictions after post processing.
    """
    sub['_x_'] = sub['x']
    sub['_y_'] = sub['y']
    sub.loc[sub['dist'] < threshold, '_x_'] = sub.loc[sub['dist'] < threshold]['x_']
    sub.loc[sub['dist'] < threshold, '_y_'] = sub.loc[sub['dist'] < threshold]['y_']
    return sub.copy()

## Floor FIx

In [ ]:
def fix_floor(site_df, paths_list, s, s_len):
    floor_fixed_df = pd.DataFrame(columns=['path','timestamp','x','y','pred_x','pred_y','fixed_x','fixed_y','f','pred_f','fixed_f'])
    
    site_df = site_df.sort_values(['path', 'timestamp'])
    
    for p, path_id in enumerate(paths_list):
        sel_path = site_df[(site_df.path == path_id)].reset_index(drop=True)
        print ("\033[A                                                                                         \033[A", end="\r")
        print(f"Site: {s:02d}/{s_len} | Fixing Predicted Floors | Path: {p+1}/{len(paths_list)} ({path_id})", end="\r")

        temp = sel_path.groupby(['pred_f'])["pred_f"].count().reset_index(name="count")
        if temp.shape[0] > 1:
            pred_floor = temp.pred_f.iloc[temp['count'].idxmax()]
        else:
            pred_floor = sel_path.pred_f.head(1).values[0]

        sel_path['fixed_f'] = pred_floor
        floor_fixed_df = floor_fixed_df.append(sel_path, ignore_index=True)
        
    return floor_fixed_df

## WP Fix

In [ ]:
def wp_fix(site_id, site_df, floors_list, paths_list, s, s_len):
    wp_fixed_df = pd.DataFrame(columns=['path','timestamp','x','y','pred_x','pred_y','fixed_x','fixed_y','f','pred_f'])
    
    site_df = site_df.sort_values(['f', 'path', 'timestamp'])
    alpha, thresshold = 2, 5.5
    betta = 0
    
    #Tunning Betta
    floor_id = 0
    floor_name = get_site_floor_from_id(site_id, floor_id)
    if floor_name==0:
        floor_id = 1
        floor_name = get_site_floor_from_id(site_id, floor_id)
        
    print ("\033[A                                                                                                                       \033[A", end="\r")
    print(f"Site: {s+1:02d}/{s_len} | Fixing Predicted XY | Floor: Tunning Betta with {floor_name} | Gererating Grid ", end="\r")
    floor_wp_grid, floor_map_grid = generate_grid_for_floor(site_id, floor_name, 2)
    sel_floor = site_df[(site_df.pred_f == floor_id)].reset_index(drop=True)     

    if betta == 0:
        min_score = 50.0
        for b in [3,4,5,6]:
            betta_eval_df = pd.DataFrame(columns=['pred_x','fixed_x'])
            for p, path_id in enumerate(paths_list):
                print ("\033[A                                                                                                                                                       \033[A", end="\r")
                print(f"Site: {s+1:02d}/{s_len} | Fixing Predicted XY | Tunning Betta ({b}/{betta}-{min_score:.2f}) | Path: {p+1}/{len(paths_list)} ({path_id})", end="\r")
                sel_path = sel_floor[(sel_floor.path == path_id)].reset_index(drop=True)
                if sel_path.empty == False:
                    knn_path = sel_path[['path','timestamp', 'pred_x', 'pred_y', 'f']]
                    knn_path = knn_path.rename({'pred_x': 'x', 'pred_y': 'y', 'f': 'floor'}, axis='columns').sort_values('timestamp')
                    knn_path['site'] = site_id

                    betta_eval = sel_path[['x','y']]
                    knn_pdr_cm_path = correct_path(site_id, knn_path, alpha, b)
                    knn_pdr_cm_wpgrid_path = snap_to_grid(knn_pdr_cm_path, floor_wp_grid, thresshold)
                    betta_eval['fixed_x'] = knn_pdr_cm_wpgrid_path['x']
                    betta_eval['fixed_y'] = knn_pdr_cm_wpgrid_path['y']
                    betta_eval_df = betta_eval_df.append(betta_eval, ignore_index=True)

            mpe = np.mean(np.sqrt(np.power(betta_eval_df.fixed_x - betta_eval_df.x, 2) + np.power(betta_eval_df.fixed_y - betta_eval_df.y, 2)))
            acc = sum(betta_eval_df.x == betta_eval_df.fixed_x)/betta_eval_df.shape[0]
            score = mpe + (1-acc)
            if score < min_score:
                min_score = score
                betta = b      
    #Fixing site floor by floor               
    for f, floor_id in enumerate(floors_list):
        floor_name = get_site_floor_from_id(site_id, floor_id)
        print ("\033[A                                                                                                                       \033[A", end="\r")
        print(f"Site: {s+1:02d}/{s_len} | Fixing Predicted XY | Floor: {f+1}/{len(floors_list)} ({floor_name}) | Gererating Grid ", end="\r")
        floor_wp_grid, floor_map_grid = generate_grid_for_floor(site_id, floor_name, 2)
        sel_floor = site_df[(site_df.pred_f == floor_id)].reset_index(drop=True)
        
        for p, path_id in enumerate(paths_list):
            print ("\033[A                                                                                                                                                       \033[A", end="\r")
            print(f"Site: {s+1:02d}/{s_len} | Fixing Predicted XY (A:{alpha},B:{betta}) | Floor: {f+1}/{len(floors_list)} ({floor_name}) | Path: {p+1}/{len(paths_list)} ({path_id})", end="\r")
            sel_path = sel_floor[(sel_floor.path == path_id)].reset_index(drop=True)

            if sel_path.empty == False:
                knn_path = sel_path[['path','timestamp', 'pred_x', 'pred_y', 'f']]
                knn_path = knn_path.rename({'pred_x': 'x', 'pred_y': 'y', 'f': 'floor'}, axis='columns').sort_values('timestamp')
                knn_path['site'] = site_id

                knn_pdr_cm_path = correct_path(site_id, knn_path, alpha, betta)
                knn_pdr_cm_wpgrid_path = snap_to_grid(knn_pdr_cm_path, floor_wp_grid, thresshold)

                sel_path['fixed_x'] = knn_pdr_cm_wpgrid_path['x']
                sel_path['fixed_y'] = knn_pdr_cm_wpgrid_path['y']
                wp_fixed_df = wp_fixed_df.append(sel_path, ignore_index=True)
    
    return wp_fixed_df           

In [ ]:
stop = stop

# RUNTIME

### KNN Analysis

In [ ]:
eval_df = pd.read_csv('../input/indoor-location-knn-kfold-predictions/knn_fold_eval.csv')
eval_df = eval_df.drop(columns=['y_acc','wp_mpe','x_acc'])
eval_df = eval_df.rename({'f_acc': 'floor_knn_acc'}, axis='columns')
sites_path  = sorted(glob.glob('../input/indoor-location-knn-kfold-predictions/5*.csv'))
wp_knn_mpe = []
wp_knn_acc = []
floor_knn_acc = []

wp_knn_rmse = []
wp_knn_xrmse = []
wp_knn_yrmse = []


for site_path in sites_path:
    site = pd.read_csv(site_path)
    wp_knn_mpe.append(np.mean(np.sqrt(np.power(site.pred_x - site.x, 2) + np.power(site.pred_y - site.y, 2))))
    
    wp_knn_acc.append(sum(site.x == site.pred_x)/site.shape[0])
    floor_knn_acc.append(sum(site.f == site.pred_f)/site.shape[0])
    
    wp_knn_xrmse.append(mean_squared_error(site.x, site.pred_x, squared=False))
    wp_knn_yrmse.append(mean_squared_error(site.y, site.pred_y, squared=False))
    
wp_knn_rmse = np.sqrt((np.power(wp_knn_xrmse, 2) + np.power(wp_knn_yrmse, 2)))    


eval_df['wp_knn_rmse'] = wp_knn_rmse
eval_df['wp_knn_mpe'] = wp_knn_mpe
eval_df['wp_knn_acc'] = wp_knn_acc
eval_df['floor_knn_acc'] = floor_knn_acc
print(f"Floor KNN Acc | Mean/Median {np.mean(eval_df.floor_knn_acc):,.2%} / {np.median(eval_df.floor_knn_acc):,.2%}  | Min/Max: {np.min(eval_df.floor_knn_acc):,.2%} / {np.max(eval_df.floor_knn_acc):,.2%}")
print(f"XY KNN MPE | Mean/Median {np.mean(eval_df.wp_knn_mpe):0.2f} / {np.median(eval_df.wp_knn_mpe):0.2f} | Min/Max: {np.min(eval_df.wp_knn_mpe):0.2f} / {np.max(eval_df.wp_knn_mpe):0.2f}")
print(f"XY KNN Acc | Mean/Median {np.mean(eval_df.wp_knn_acc):,.2%} / {np.median(eval_df.wp_knn_acc):,.2%} | Min/Max: {np.min(eval_df.wp_knn_acc):,.2%} / {np.max(eval_df.wp_knn_acc):,.2%}")
display(eval_df)

In [ ]:
#WP RMSE
fig_wp_rmse = go.Figure(data=[
    go.Bar(name='Predicted', x=eval_df.index.astype(str), y=eval_df.wp_knn_rmse,text=eval_df.wp_knn_rmse,textposition='auto'),
])
fig_wp_rmse.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig_wp_rmse.update_layout(legend=dict(yanchor="top",y=0.95,xanchor="left",x=0.01), barmode='group', xaxis_tickangle=0)

#WP MPE
fig_wp_mpe = go.Figure(data=[
    go.Bar(name='Predicted', x=eval_df.index.astype(str), y=eval_df.wp_knn_mpe,text=eval_df.wp_knn_mpe,textposition='auto'),
])
fig_wp_mpe.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig_wp_mpe.update_layout(legend=dict(yanchor="top",y=0.95,xanchor="left",x=0.01), barmode='group', xaxis_tickangle=0)

#WP ACC
fig_wp_acc = go.Figure(data=[
    go.Bar(name='Predicted', x=eval_df.index.astype(str), y=eval_df.wp_knn_acc,text=eval_df.wp_knn_acc,textposition='auto')
])
fig_wp_acc.update_traces(texttemplate='%{text:,.0%}', textposition='outside')
fig_wp_acc.update_layout(yaxis=dict(tickformat=".0%"))
fig_wp_acc.update_layout(legend=dict(yanchor="top",y=0.25,xanchor="left",x=0.01))
#Floor ACC
fig_f_acc = go.Figure(data=[
    go.Bar(name='Predicted', x=eval_df.index.astype(str), y=eval_df.floor_knn_acc,text=eval_df.floor_knn_acc,textposition='auto'),
])
fig_f_acc.update_traces(texttemplate='%{text:,.0%}', textposition='outside')
fig_f_acc.update_layout(yaxis=dict(tickformat=".0%"))
fig_f_acc.update_layout(legend=dict(yanchor="top",y=0.25,xanchor="left",x=0.01))

fig_wp_rmse.show()
fig_wp_mpe.show()
fig_wp_acc.show()
fig_f_acc.show()

In [ ]:
eval_df = pd.read_csv('../input/mcs-submissions/final_fixe_eval.csv')
print(f"Floor KNN Acc | Mean/Median {np.mean(eval_df.floor_knn_acc):,.2%} / {np.median(eval_df.floor_knn_acc):,.2%}  | Min/Max: {np.min(eval_df.floor_knn_acc):,.2%} / {np.max(eval_df.floor_knn_acc):,.2%}")
print(f"Floor FIX Acc | Mean/Median {np.mean(eval_df.floor_fix_acc):,.2%} / {np.median(eval_df.floor_fix_acc):,.2%}  | Min/Max: {np.min(eval_df.floor_fix_acc):,.2%} / {np.max(eval_df.floor_fix_acc):,.2%}\n")

print(f"XY KNN MPE | Mean/Median {np.mean(eval_df.wp_knn_mpe):0.2f} / {np.median(eval_df.wp_knn_mpe):0.2f} | Min/Max: {np.min(eval_df.wp_knn_mpe):0.2f} / {np.max(eval_df.wp_knn_mpe):0.2f}")
print(f"XY FIX MPE | Mean/Median {np.mean(eval_df.wp_fix_mpe):0.2f} / {np.median(eval_df.wp_fix_mpe):0.2f} | Min/Max: {np.min(eval_df.wp_fix_mpe):0.2f} / {np.max(eval_df.wp_fix_mpe):0.2f}")
print(f"XY GRID MPE | Mean/Median {np.mean(eval_df.wp_grid_mpe):0.2f} / {np.median(eval_df.wp_grid_mpe):0.2f} | Min/Max: {np.min(eval_df.wp_grid_mpe):0.2f} / {np.max(eval_df.wp_grid_mpe):0.2f}\n")

print(f"XY KNN Acc | Mean/Median {np.mean(eval_df.wp_knn_acc):,.2%} / {np.median(eval_df.wp_knn_acc):,.2%} | Min/Max: {np.min(eval_df.wp_knn_acc):,.2%} / {np.max(eval_df.wp_knn_acc):,.2%}")
print(f"XY FIX Acc | Mean/Median {np.mean(eval_df.wp_fix_acc):,.2%} / {np.median(eval_df.wp_fix_acc):,.2%} | Min/Max: {np.min(eval_df.wp_fix_acc):,.2%} / {np.max(eval_df.wp_fix_acc):,.2%}")
display(eval_df.style.apply(highlight_eval, axis=None))

In [ ]:
fig = go.Figure(data=[go.Bar(x=eval_df.index.astype(str), y=eval_df.wp_knn_mpe,text=eval_df.wp_knn_mpe,textposition='auto',)])
#fig.update_traces(texttemplate='%{text:,.0%}', textposition='outside')
#fig.update_layout(yaxis=dict(tickformat=".0%"))

fig.update_traces(texttemplate='%{text:0.2f}', textposition='outside')
fig.show()

# POST-PROCESSING

In [ ]:
#fixed_df = pd.DataFrame(columns=['path','timestamp','x','y','pred_x','pred_y','f','pred_f','fixed_f']) 
pd.options.mode.chained_assignment = None
wp_fix_mpe = []
wp_fix_acc = []
floor_fix_acc = []
wp_fix_rmse = []

In [ ]:
sites_path = sites_path[22:24]

In [ ]:

for s, site_path in enumerate(sites_path):
    
    #load
    site_id = site_path.split('/')[-1].split('.')[0]
    site_df = pd.read_csv(site_path)
    paths_list = site_df.groupby('path')['path'].count().reset_index(name="count").path.to_numpy()
    floors_list = site_df.groupby('pred_f')['pred_f'].count().reset_index(name="count").pred_f.to_numpy()
    
    #fix
    wp_fixed_df = wp_fix(site_id, site_df, floors_list, paths_list, s, len(sites_path))
    floor_fixed_df = fix_floor(wp_fixed_df, paths_list, s+1, len(sites_path))
    
    outdir = './fixed_predictions'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    wp_fixed_df.to_csv('./fixed_predictions/{}.csv'.format(site_id),index=False)
    
    #evaluate
    fixed_df = floor_fixed_df.copy()
    wp_fix_mpe.append(np.mean(np.sqrt(np.power(fixed_df.fixed_x - fixed_df.x, 2) + np.power(fixed_df.fixed_y - fixed_df.y, 2))))
    wp_fix_acc.append(sum(fixed_df.x == fixed_df.fixed_x)/fixed_df.shape[0])
    
    wp_fix_xrmse = mean_squared_error(fixed_df.x, fixed_df.fixed_x, squared=False)
    wp_fix_yrmse = mean_squared_error(fixed_df.y, fixed_df.fixed_y, squared=False)
    wp_fix_rmse.append(np.sqrt((np.power(wp_fix_xrmse, 2) + np.power(wp_fix_yrmse, 2))))
    
    #add score
    floor_fix_acc.append(sum(fixed_df.f == fixed_df.fixed_f)/fixed_df.shape[0])

eval_df['floor_fix_acc'] = floor_fix_acc
eval_df['wp_fix_rmse'] = wp_fix_rmse
eval_df['wp_fix_mpe'] = wp_fix_mpe
eval_df['wp_fix_acc'] = wp_fix_acc

In [ ]:
eval_df['floor_fix_acc'] = floor_fix_acc
eval_df['wp_fix_rmse'] = wp_fix_rmse
eval_df['wp_fix_mpe'] = wp_fix_mpe
eval_df['wp_fix_acc'] = wp_fix_acc

# EVALUATIONS

In [ ]:
#eval_df = pd.read_csv('../input/mcs-submissions/final_fixe_eval.csv')
print(f"Floor KNN Acc | Mean/Median {np.mean(eval_df.floor_knn_acc):,.2%} / {np.median(eval_df.floor_knn_acc):,.2%}  | Min/Max: {np.min(eval_df.floor_knn_acc):,.2%} / {np.max(eval_df.floor_knn_acc):,.2%}")
print(f"Floor FIX Acc | Mean/Median {np.mean(eval_df.floor_fix_acc):,.2%} / {np.median(eval_df.floor_fix_acc):,.2%}  | Min/Max: {np.min(eval_df.floor_fix_acc):,.2%} / {np.max(eval_df.floor_fix_acc):,.2%}\n")

print(f"XY KNN RSME | Mean/Median {np.mean(eval_df.wp_knn_rmse):0.2f} / {np.median(eval_df.wp_knn_rmse):0.2f} | Min/Max: {np.min(eval_df.wp_knn_rmse):0.2f} / {np.max(eval_df.wp_knn_rmse):0.2f}")
print(f"XY FIX RSME | Mean/Median {np.mean(eval_df.wp_fix_rmse):0.2f} / {np.median(eval_df.wp_fix_rmse):0.2f} | Min/Max: {np.min(eval_df.wp_fix_rmse):0.2f} / {np.max(eval_df.wp_fix_rmse):0.2f}\n")

print(f"XY KNN MDE | Mean/Median {np.mean(eval_df.wp_knn_mpe):0.2f} / {np.median(eval_df.wp_knn_mpe):0.2f} | Min/Max: {np.min(eval_df.wp_knn_mpe):0.2f} / {np.max(eval_df.wp_knn_mpe):0.2f}")
print(f"XY FIX MDE | Mean/Median {np.mean(eval_df.wp_fix_mpe):0.2f} / {np.median(eval_df.wp_fix_mpe):0.2f} | Min/Max: {np.min(eval_df.wp_fix_mpe):0.2f} / {np.max(eval_df.wp_fix_mpe):0.2f}\n")

print(f"XY KNN Acc | Mean/Median {np.mean(eval_df.wp_knn_acc):,.2%} / {np.median(eval_df.wp_knn_acc):,.2%} | Min/Max: {np.min(eval_df.wp_knn_acc):,.2%} / {np.max(eval_df.wp_knn_acc):,.2%}")
print(f"XY FIX Acc | Mean/Median {np.mean(eval_df.wp_fix_acc):,.2%} / {np.median(eval_df.wp_fix_acc):,.2%} | Min/Max: {np.min(eval_df.wp_fix_acc):,.2%} / {np.max(eval_df.wp_fix_acc):,.2%}")
display(eval_df.style.apply(highlight_eval, axis=None))

In [ ]:
rmse_dec = 100- eval_df.wp_fix_rmse*100/eval_df.wp_knn_rmse
rmse_dec[1]
l1 = [ti for ti in eval_df.index.astype(str)]
l2 = [f'(↘{ti:.0f}%)' for ti in rmse_dec]
annotationsList = [l1,l2]
#WP MPE
fig_wp_rmse = go.Figure(data=[
    go.Bar(name='Predicted', x=annotationsList, y=eval_df.wp_knn_rmse,text=eval_df.wp_knn_rmse,textposition='auto'),
    go.Bar(name='Fixed', x=annotationsList, y=eval_df.wp_fix_rmse,text=eval_df.wp_fix_rmse,textposition='auto')
])
fig_wp_rmse.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig_wp_rmse.update_layout(legend=dict(yanchor="top",y=0.95,xanchor="left",x=0.01), barmode='group', xaxis_tickangle=0)

mpe_dec = 100- eval_df.wp_fix_mpe*100/eval_df.wp_knn_mpe
mpe_dec[1]
l1 = [ti for ti in eval_df.index.astype(str)]
l2 = [f'(↘{ti:.0f}%)' for ti in mpe_dec]
annotationsList = [l1,l2]
#WP MPE
fig_wp_mpe = go.Figure(data=[
    go.Bar(name='Predicted', x=annotationsList, y=eval_df.wp_knn_mpe,text=eval_df.wp_knn_mpe,textposition='auto'),
    go.Bar(name='Fixed', x=annotationsList, y=eval_df.wp_fix_mpe,text=eval_df.wp_fix_mpe,textposition='auto')
])
fig_wp_mpe.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig_wp_mpe.update_layout(legend=dict(yanchor="top",y=0.95,xanchor="left",x=0.01), barmode='group', xaxis_tickangle=0)

#WP ACC
fig_wp_acc = go.Figure(data=[
    go.Bar(name='Predicted', x=eval_df.index.astype(str), y=eval_df.wp_knn_acc,text=eval_df.wp_knn_acc,textposition='auto'),
    go.Bar(name='Fixed', x=eval_df.index.astype(str), y=eval_df.wp_fix_acc,text=eval_df.wp_fix_acc,textposition='auto')
    
])
fig_wp_acc.update_traces(texttemplate='%{text:,.0%}', textposition='outside')
fig_wp_acc.update_layout(yaxis=dict(tickformat=".0%"))
fig_wp_acc.update_layout(legend=dict(yanchor="top",y=0.25,xanchor="left",x=0.01))
#Floor ACC
fig_f_acc = go.Figure(data=[
    go.Bar(name='Predicted', x=eval_df.index.astype(str), y=eval_df.floor_knn_acc,text=eval_df.floor_knn_acc,textposition='auto'),
    go.Bar(name='Fixed', x=eval_df.index.astype(str), y=eval_df.floor_fix_acc,text=eval_df.floor_fix_acc,textposition='auto')
])
fig_f_acc.update_traces(texttemplate='%{text:,.0%}', textposition='outside')
fig_f_acc.update_layout(yaxis=dict(tickformat=".0%"))
fig_f_acc.update_layout(legend=dict(yanchor="top",y=0.25,xanchor="left",x=0.01))

fig_wp_rmse.show()
fig_wp_mpe.show()
fig_wp_acc.show()
fig_f_acc.show()

In [ ]:
eval_df_1 = pd.read_csv('../input/mcs-submissions/fixed_eval_1.csv')

In [ ]:
print(f"Floor KNN Acc | Mean/Median {np.mean(eval_df_1.floor_knn_acc):,.2%} / {np.median(eval_df_1.floor_knn_acc):,.2%}  | Min/Max: {np.min(eval_df_1.floor_knn_acc):,.2%} / {np.max(eval_df_1.floor_knn_acc):,.2%}")
print(f"Floor FIX Acc | Mean/Median {np.mean(eval_df_1.floor_fix_acc):,.2%} / {np.median(eval_df_1.floor_fix_acc):,.2%}  | Min/Max: {np.min(eval_df_1.floor_fix_acc):,.2%} / {np.max(eval_df_1.floor_fix_acc):,.2%}\n")

print(f"XY KNN MPE | Mean/Median {np.mean(eval_df_1.wp_knn_mpe):0.2f} / {np.median(eval_df_1.wp_knn_mpe):0.2f} | Min/Max: {np.min(eval_df_1.wp_knn_mpe):0.2f} / {np.max(eval_df_1.wp_knn_mpe):0.2f}")
print(f"XY FIX MPE | Mean/Median {np.mean(eval_df_1.wp_fix_mpe):0.2f} / {np.median(eval_df_1.wp_fix_mpe):0.2f} | Min/Max: {np.min(eval_df_1.wp_fix_mpe):0.2f} / {np.max(eval_df_1.wp_fix_mpe):0.2f}\n")

print(f"XY KNN Acc | Mean/Median {np.mean(eval_df_1.wp_knn_acc):,.2%} / {np.median(eval_df_1.wp_knn_acc):,.2%} | Min/Max: {np.min(eval_df_1.wp_knn_acc):,.2%} / {np.max(eval_df_1.wp_knn_acc):,.2%}")
print(f"XY FIX Acc | Mean/Median {np.mean(eval_df_1.wp_fix_acc):,.2%} / {np.median(eval_df_1.wp_fix_acc):,.2%} | Min/Max: {np.min(eval_df_1.wp_fix_acc):,.2%} / {np.max(eval_df_1.wp_fix_acc):,.2%}\n")

display(eval_df_1.style.apply(highlight_eval, axis=None))

In [ ]:
#WP MPE
fig_wp_mpe = go.Figure(data=[
    go.Bar(name='Predicted', x=eval_df_1.index.astype(str), y=eval_df_1.wp_knn_mpe,text=eval_df_1.wp_knn_mpe,textposition='auto'),
    go.Bar(name='Fixed', x=eval_df_1.index.astype(str), y=eval_df_1.wp_fix_mpe,text=eval_df_1.wp_fix_mpe,textposition='auto')
])
fig_wp_mpe.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig_wp_mpe.update_layout(legend=dict(yanchor="top",y=0.95,xanchor="left",x=0.01))
#WP ACC
fig_wp_acc = go.Figure(data=[
    go.Bar(name='Predicted', x=eval_df_1.index.astype(str), y=eval_df_1.wp_knn_acc,text=eval_df_1.wp_knn_acc,textposition='auto'),
    go.Bar(name='Fixed', x=eval_df_1.index.astype(str), y=eval_df_1.wp_fix_acc,text=eval_df_1.wp_fix_acc,textposition='auto')
])
fig_wp_acc.update_traces(texttemplate='%{text:,.0%}', textposition='outside')
fig_wp_acc.update_layout(yaxis=dict(tickformat=".0%"))
fig_wp_acc.update_layout(legend=dict(yanchor="top",y=0.25,xanchor="left",x=0.01))
#Floor ACC
fig_f_acc = go.Figure(data=[
    go.Bar(name='Predicted', x=eval_df_1.index.astype(str), y=eval_df_1.floor_knn_acc,text=eval_df_1.floor_knn_acc,textposition='auto'),
    go.Bar(name='Fixed', x=eval_df_1.index.astype(str), y=eval_df_1.floor_fix_acc,text=eval_df_1.floor_fix_acc,textposition='auto')
])
fig_f_acc.update_traces(texttemplate='%{text:,.0%}', textposition='outside')
fig_f_acc.update_layout(yaxis=dict(tickformat=".0%"))
fig_f_acc.update_layout(legend=dict(yanchor="top",y=0.25,xanchor="left",x=0.01))

fig_wp_mpe.show()
fig_wp_acc.show()
fig_f_acc.show()

# Example Visualization

In [ ]:
#../input/indoor-location-knn-kfold-predictions/5a0546857ecc773753327266.csv

In [ ]:
site_sample_path = '../input/indoor-location-knn-kfold-predictions/5c3c44b80379370013e0fd2b.csv'
site_to_check = site_sample_path.split('/')[-1].split('.')[0]
site_df = pd.read_csv(site_sample_path)

In [ ]:
sample_df = site_df[['x','pred_x']].sort_values('pred_x')
y_range = list(range(1, len(sample_df.x)))
stdev = mean_squared_error(sample_df.x, sample_df.pred_x, squared=False)
fig3 = go.Figure()
fig3.add_trace(go.Scatter(y=sample_df.x, x=y_range, mode='markers', name='Original'))
fig3.add_trace(go.Scatter(y=sample_df.pred_x, x=y_range, mode='lines', name='Predictions', line_shape='spline'))
fig3.add_trace(go.Scatter(y=sample_df.pred_x+stdev, x=y_range, mode='lines', name='Residual Standard Deviation', line = dict(color = 'green', dash = 'dash')))
fig3.add_trace(go.Scatter(y=sample_df.pred_x-stdev, x=y_range, mode='lines', name='RMSE', showlegend=False, line = dict(color = 'green', dash = 'dash')))


fig3.show()

In [ ]:
floor_to_check = get_site_floor_from_id(site_to_check, rand_path.pred_f.sample(1).values[0])
floor_paths_df, floor_wp_grid, floor_map_grid = generate_grid_for_floor(site_to_check, floor_to_check, 3, True)
print('GRID GENERATING IS COMPLITE')

In [ ]:
rand_path_id = site_df.path[site_df.pred_f == 0].sample(1).values[0]
#rand_path_id = '5e15bf941506f2000638fec5'
#rand_path_id = '5dd2616e44333f00067a9979' #all wrong wp fixed, site 5d27097f03f801723c320d97
rand_path = site_df[site_df.path == rand_path_id].sort_values('timestamp')
display(rand_path.style.apply(highlight_dif, axis=None))

In [ ]:
pd.options.mode.chained_assignment = None
knn_path = rand_path.drop(columns=['x', 'y', 'f'])
knn_path = knn_path.rename({'pred_x': 'x', 'pred_y': 'y', 'pred_f': 'floor'}, axis='columns').sort_values('timestamp')
knn_path['site'] = site_to_check

temp = knn_path.copy()
knn_pdr_cm_path = correct_path(site_to_check, temp, 2, 3)

temp = knn_path.copy()
temp = snap_to_grid(temp, floor_wp_grid, 2)
knn_wpgrid_path = temp

temp = knn_path.copy()
temp = snap_to_grid(temp, floor_map_grid, 5)
knn_mapgrid_path = temp

temp = snap_to_grid(knn_pdr_cm_path, floor_wp_grid, 5)
knn_pdr_cm_wpgrid_path = temp

temp = snap_to_grid(knn_pdr_cm_path, floor_map_grid, 5)
knn_pdr_cm_mapgrid_path = temp

original_path = floor_paths_df[floor_paths_df.path == rand_path_id]
original_path['path'] = '1.Original'
knn_path['path'] = '2.KNN only'
knn_pdr_cm_path['path'] = '3.KNN+PDR+CM'
knn_pdr_cm_wpgrid_path['path'] = '4.KNN+PDR+CM+WP-Grid'

knn_wpgrid_path['path'] = '6.1.KNN+WP-Grid'
knn_mapgrid_path['path'] = '6.1.KNN+Gen-Grid'
knn_pdr_cm_mapgrid_path['path'] = '6.3.KNN+PDR+CM+Gen-Grid'

compare_paths = original_path.append([knn_path, knn_pdr_cm_path, knn_wpgrid_path, knn_mapgrid_path, knn_pdr_cm_wpgrid_path, knn_pdr_cm_mapgrid_path], ignore_index=True)
del compare_paths['xy']
#display(compare_paths)
print('POST-PROCESSING IS COMPLITE')

In [ ]:
plot_floor(site_to_check, floor_to_check, compare_paths, floor_wp_grid, floor_map_grid).show()

In [ ]:
sites_path

In [ ]:
all_map_path

In [ ]:
all_map_path = sites_path[22:24]
for s, site_path in enumerate(all_map_path):
    print(site_path)
    site_id = site_path.split('/')[-1].split('.')[0]
    floor_to_check = get_site_floor_from_id(site_id, 0)
    if floor_to_check == 0:
        floor_to_check = get_site_floor_from_id(site_id, 1)
    print(s, site_id, floor_to_check)
    floor_wp_grid, floor_map_grid = generate_grid_for_floor(site_id, floor_to_check, 4)
    plot_floor(site_id, floor_to_check, None, floor_wp_grid, floor_map_grid).show()